In [ ]:
import pandas as pd
import plotly.graph_objects as go
import datetime
import numpy as np
from copy import deepcopy
import cProfile
import sys
from collections import defaultdict
from tqdm import tqdm 
import pickle
from bisect import bisect_left
import random
from tqdm import tqdm
import random
import logging


from graph import TransportGraph, ContactionTransportGraph
from ttf import TTF
from forward_search import FCH
from dijkstra import Dijkstra

# Build transport graph

In [ ]:
CITY = 'kuopio'

## Build Transport Graph

In [ ]:
transport_connections = pd.read_csv(F'data/{CITY}/network_temporal_day.csv', sep=';')
walk_connections = pd.read_csv(F'data/{CITY}/network_walk.csv', sep=';')

In [ ]:
df_walk_invert = walk_connections.copy()
df_walk_invert = df_walk_invert.rename(columns={'from_stop_I': 'to_stop_I', 'to_stop_I': 'from_stop_I'})
walk_connections = pd.concat((walk_connections, df_walk_invert))

In [ ]:
tg = TransportGraph(transport_connections=transport_connections, walk_connections=walk_connections)

# Convert ATF to TTF

In [ ]:
tg_ttf = deepcopy(tg)
for node1, out in tg_ttf.graph.items():
    for node2, f in out.items():
        tg_ttf.graph[node1][node2] = TTF(f)

## Graph statistics

In [ ]:
tg.edges_cnt, tg.nodes_cnt, tg.timetable_stats

### Save TG with ATF and TG with TTF

In [ ]:
pickle.dump(tg, open(F'{CITY}.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)
pickle.dump(tg_ttf, open(F'{CITY}_ttf.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)

# Build CH graph

In [ ]:
%%time
cProfile.run('ch_tg = tg.contraction_hierarchy()')

## Graph statistics

In [ ]:
ch_tg.edges_cnt, ch_tg.nodes_cnt, ch_tg.timetable_stats

### Calculate Geometrical Containers for speed up future search

In [ ]:
ch_tg.geometrical_container()

# Convert ATF to TTF in CH-graph

In [ ]:
ch_tg_ttf = deepcopy(ch_tg)
for node1, out in ch_tg_ttf.graph.items():
    for node2, f in out.items():
        ch_tg_ttf.graph[node1][node2] = TTF(f)

## Save graph

In [ ]:
pickle.dump(ch_tg, open(F'{CITY}_ch.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)
pickle.dump(ch_tg, open(F'{CITY}_ch_ttf.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)

# Calulate TTN for TG AND CH-graph

In [ ]:
tg.optimize_binary_search()

In [ ]:
ch_tg.optimize_binary_search()

# Compare solutions

In [ ]:
N = 1000
test_data = pd.DataFrame({'start_time': [random.randint(transport_connections['dep_time_ut'].min(), 
                                           transport_connections['dep_time_ut'].max()) for i in range(N)],
             'start_node' : [random.sample(tg.nodes, 1)[0] for i in range(N)], 
              'end_node' : [random.sample(tg.nodes, 1)[0] for i in range(N)]
             })

algorithms = ['fch', 'fch_ttf', 'fch_binary_duration', 'dijkstra', 'dijkstra_binary_duration', 'dijkstra_ttf']
duration = {x: [] for x in algorithms}
arrival = {x: 0 for x in algorithms}
for index, row in test_data.iterrows():
    random.shuffle(algorithms)
    for algorithm in algorithms:
        if algorithm == 'fch':
            pathfinding = FCH(graph=ch_tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=False, 
                                             next_index_optimization=False)
            if path['path']:
                duration[algorithm].append(path['duration'])
            arrival[algorithm] = path['arrival']
        elif algorithm == 'fch_ttf':
            pathfinding = FCH(graph=ch_tg_ttf,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=False, 
                                             next_index_optimization=False)
            if path['path']:
                duration[algorithm].append(path['duration'])
            arrival[algorithm] = path['arrival']
        elif algorithm == 'fch_binary_duration':
            pathfinding = FCH(graph=ch_tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=True,
                                             next_index_optimization=False)
            if path['path']:
                duration[algorithm].append(path['duration'])
            arrival[algorithm] = path['arrival']
            
        elif algorithm == 'dijkstra':
            pathfinding = Dijkstra(graph=tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=False)
            if path['path']:
                duration[algorithm].append(path['duration'])
            arrival[algorithm] = path['arrival']
        elif algorithm == 'dijkstra_ttf':
            pathfinding = Dijkstra(graph=tg_ttf,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=False)
            if path['path']:
                duration[algorithm].append(path['duration'])
            arrival[algorithm] = path['arrival']
        elif algorithm == 'dijkstra_binary_duration':
            pathfinding = Dijkstra(graph=tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=True)
            if path['path']:
                duration[algorithm].append(path['duration'])
            arrival[algorithm] = path['arrival']
    arr = arrival[algorithms[0]]        
    for i in range(1, len(algorithms)):
        assert arr == arrival[algorithms[i]]  

In [ ]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.median(duration[algorithm]), np.std(duration[algorithm]))

In [ ]:
from bisect import bisect_left

for node1, out in tqdm(ch_tg.graph.items()):
    for node2, f in out.items():
        if ch_tg.hierarchy[node2] > ch_tg.hierarchy[node2]:
            for i, bus in enumerate(f.buses):
                
                upper_index = bisect_left(ch_tg.nodes_schedule[node2], bus.a)
    
                if i == 0:
                    bus.lower_index = 0
                else:
                   bus.lower_index = bisect_left(ch_tg.nodes_schedule[node2], f.buses[i - 1].a)

                bus.next_nodes_schedule = ch_tg.nodes_schedule[node2][bus.lower_index:upper_index]
        else:
            for i, bus in enumerate(f.buses):
                upper_index = bisect_left(ch_tg.nodes_schedule_down[node2], bus.a)
    
                if i == 0:
                    bus.lower_index = 0
                else:
                   bus.lower_index = bisect_left(ch_tg.nodes_schedule_down[node2], f.buses[i - 1].a)

                bus.next_nodes_schedule = ch_tg.nodes_schedule_down[node2][bus.lower_index:upper_index]

# Results

In [ ]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.median(duration[algorithm]), np.std(duration[algorithm]))